In [1]:
# The model that you want to train from the Hugging Face hub
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"

# The instruction dataset to use
dataset_name = "pechaut/cairo-instruct"

# Fine-tuned model name
new_model = "Mistral-7b-instruct-cairo-test"

# Output directory where the model predicti
# ons and checkpoints will be stored
output_dir = "./results"

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

hub_name = "pechaut/Mistral-7b-instruct-cairo-instruct"

max_steps = 700 # to tweak to get the best out of the model 

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,GPTQConfig
 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)

# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name,
                                             trust_remote_code=True,
                                             revision="main",
                                             quantization_config=quantization_config_loading,
                                             device_map={"": 0}
                                             )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [21]:
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


Training

In [22]:
from datasets import load_dataset

# Load the dataset
dataset_train = load_dataset(dataset_name, split="train", download_mode='force_redownload',ignore_verifications=True)
dataset_test = load_dataset(dataset_name, split="eval", download_mode='force_redownload',ignore_verifications=True)

/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/datasets/load.py:2097: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/197 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating eval split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/197 [00:00<?, ? examples/s]

In [23]:
from pickle import FALSE


peft_config = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, peft_config)
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=2.5e-5,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_dir="./logs", 
    logging_steps=5,
    num_train_epochs=1,
    max_steps=max_steps,
    fp16=True,
    push_to_hub=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=512
)


Map:   0%|          | 0/197 [00:00<?, ? examples/s]

/home/pec/miniconda3/envs/cairo-code/lib/python3.9/site-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [24]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(model)

trainable params: 3407872 || all params: 265818112 || trainable%: 1.2820315268810576


In [25]:

trainer.train()
trainer.push_to_hub()


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


AssertionError: No inf checks were recorded for this optimizer.

In [29]:

import pandas as pd
df = pd.DataFrame(dataset_train)

# Vérifier la présence de NaN dans la colonne 'text'
nan_count = df['text'].isna().sum()

if nan_count > 0:
    print(f"Il y a {nan_count} valeurs NaN dans la colonne 'text'.")
else:
    print("Il n'y a pas de valeurs NaN dans la colonne 'text'.")

Il n'y a pas de valeurs NaN dans la colonne 'text'.
